In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, DirGNNConv
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
import random, h5py, pickle, glob, os
import networkx as nx
from copy import deepcopy

output_dir = r"C:\Users\uhewm\Desktop\ProjectHGT\simulation_chunks(4)"
all_files = sorted(glob.glob(os.path.join(output_dir, "*.h5")))

graphs = []
graphs_modified = []

for file in random.sample(all_files, 1000):
    with h5py.File(file, "r") as f:
        grp = f["results"]
        graph_properties = pickle.loads(grp["graph_properties"][()])
        
        nodes = torch.tensor(graph_properties[0])              # [num_nodes]
        edges = torch.tensor(graph_properties[1], dtype=torch.long)  # [2, num_edges]
        coords = torch.tensor(graph_properties[2].T)           # [2, num_nodes]
        #edges_reversed = edges.flip(0)  # Tauscht Zeile 0 und 1

        x_node_features = coords.float().T
        
        # Erstelle einen gerichteten Graphen
        G = nx.DiGraph()
        
        # Füge Knoten hinzu (optional mit Koordinaten als Attribut)
        for i, node_id in enumerate(nodes.tolist()):
            G.add_node(node_id, core_distance = coords[:, i].tolist()[0], allele_distance = coords[:, i].tolist()[1])
        
        # Füge Kanten hinzu
        edge_list = edges.tolist()
        for src, dst in zip(edge_list[0], edge_list[1]):
            G.add_edge(src, dst)

        H = deepcopy(G)
            
        for node in G.nodes():
            children = list(G.predecessors(node))
            if children:
                core_sum = sum(G.nodes[child]['core_distance'] for child in children)
                allele_sum = sum(G.nodes[child]['allele_distance'] for child in children)
                H.nodes[node]['core_distance'] = G.nodes[node]['core_distance'] - core_sum
                H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance'] - allele_sum
            else:
                H.nodes[node]['core_distance'] = G.nodes[node]['core_distance']
                H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance']


        node_features = []
        for node in list(H.nodes):
            core = H.nodes[node].get("core_distance", 0.0)
            allele = H.nodes[node].get("allele_distance", 0.0)
            node_features.append([core, allele])
        coords_modified = torch.tensor(node_features, dtype=torch.float32).T
         
        # Node features: nur die zwei Werte pro Knoten (coords)
        x_node_features_modified = coords_modified.float().T  # Shape [num_nodes, 2]

        # Labels
        theta_gains = torch.tensor(
            [1 if node in grp.attrs["parental_nodes_hgt_events_corrected"] else 0 
             for node in graph_properties[0]],
            dtype=torch.long
        )

        # PyG-Graph erstellen
        data = Data(
            x=x_node_features,       # Node Features [num_nodes, 2]
            edge_index=edges,        # Edge Index [2, num_edges]
            y=theta_gains            # Labels [num_nodes]
        )
        graphs.append(data)

        # PyG-Graph erstellen
        data_modified = Data(
            x=x_node_features_modified,       # Node Features [num_nodes, 2]
            edge_index=edges,        # Edge Index [2, num_edges]
            y=theta_gains            # Labels [num_nodes]
        )
        graphs_modified.append(data_modified)


C:\Users\uhewm\AppData\Local\Temp\ipykernel_11436\406062616.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1], dtype=torch.long)  # [2, num_edges]
C:\Users\uhewm\AppData\Local\Temp\ipykernel_11436\406062616.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(graph_properties[2].T)           # [2, num_nodes]


In [59]:
graphs = graphs_modified

In [60]:
#### FUNKTIONIERT!

from torch_geometric.nn import GCNConv, DirGNNConv, GATConv

graphs = graphs_modified

# Train/Test Split
random.shuffle(graphs)
split_idx = int(0.8 * len(graphs))
train_graphs = graphs[:split_idx]
test_graphs = graphs[split_idx:]

train_loader = DataLoader(train_graphs, batch_size=8, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=8)

"""
# === 2. Modell definieren ===
class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, dropout=0.3):
        super().__init__()
        # Innerer conv wird an DirGNNConv übergeben
        self.conv1 = DirGNNConv(GCNConv(in_channels, hidden_channels))
        self.conv2 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x.view(-1)
"""

import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, dropout=0.3):
        super().__init__()
        self.conv1 = DirGNNConv(GCNConv(in_channels, hidden_channels), alpha = 0)
        self.conv2 = DirGNNConv(GCNConv(hidden_channels, hidden_channels), alpha = 0)
        self.lin = nn.Linear(hidden_channels, 1)
        self.dropout = dropout

    def forward(self, x, edge_index):
        #edge_index =  torch.cat([edge_index, edge_index.flip(0)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x.view(-1)


# === 3. Modell, Optimizer, Loss ===
model = GCNClassifier(in_channels=2, hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in train_graphs])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 4. Training & Evaluation ===
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > 0.5
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        # Metriken für Klasse 1
        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

# === 5. Training starten ===
for epoch in range(1, 51):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

C:\Users\uhewm\AppData\Local\Temp\ipykernel_11436\91557288.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Pos Weight: 7.02
Epoch 01 | Loss: 0.6993 | Acc: 0.975 | Prec: 0.422 | Rec: 0.894 | F1: 0.574
Epoch 02 | Loss: 0.3453 | Acc: 0.981 | Prec: 0.488 | Rec: 0.912 | F1: 0.636
Epoch 03 | Loss: 0.1872 | Acc: 0.985 | Prec: 0.559 | Rec: 0.894 | F1: 0.688
Epoch 04 | Loss: 0.1484 | Acc: 0.987 | Prec: 0.602 | Rec: 0.878 | F1: 0.715
Epoch 05 | Loss: 0.1324 | Acc: 0.986 | Prec: 0.575 | Rec: 0.912 | F1: 0.705
Epoch 06 | Loss: 0.1179 | Acc: 0.985 | Prec: 0.551 | Rec: 0.905 | F1: 0.685
Epoch 07 | Loss: 0.1108 | Acc: 0.985 | Prec: 0.553 | Rec: 0.878 | F1: 0.678
Epoch 08 | Loss: 0.1070 | Acc: 0.990 | Prec: 0.667 | Rec: 0.874 | F1: 0.756
Epoch 09 | Loss: 0.1063 | Acc: 0.991 | Prec: 0.707 | Rec: 0.836 | F1: 0.766
Epoch 10 | Loss: 0.1009 | Acc: 0.987 | Prec: 0.607 | Rec: 0.886 | F1: 0.721
Epoch 11 | Loss: 0.0959 | Acc: 0.987 | Prec: 0.607 | Rec: 0.905 | F1: 0.727
Epoch 12 | Loss: 0.0945 | Acc: 0.987 | Prec: 0.608 | Rec: 0.902 | F1: 0.727
Epoch 13 | Loss: 0.0951 | Acc: 0.989 | Prec: 0.660 | Rec: 0.878 | F1: 0

In [61]:
import numpy as np
from torch_geometric.utils import add_self_loops

# === 2. Modell definieren ===
class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, dropout=0.3):
        super().__init__()
        self.conv1 = DirGNNConv(GCNConv(in_channels, hidden_channels), alpha = 0)
        self.conv2 = DirGNNConv(GCNConv(hidden_channels, hidden_channels), alpha = 0)
        self.conv3 = DirGNNConv(GCNConv(hidden_channels, hidden_channels), alpha = 0)
        self.conv4 = DirGNNConv(GCNConv(hidden_channels, hidden_channels), alpha = 0)
        self.conv5 = DirGNNConv(GCNConv(hidden_channels, hidden_channels), alpha = 0)
        self.lin = nn.Linear(hidden_channels, 1)
        self.dropout = dropout

    def forward(self, x, edge_index):
        #edge_index = edge_index[[1, 0], :]
        edge_index, _ = add_self_loops(edge_index)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)


        x = self.conv5(x, edge_index)
        x = F.relu(x)

        x = self.lin(x)
        return x.view(-1)


# === 3. Modell, Optimizer, Loss ===
model = GCNClassifier(in_channels=2, hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in train_graphs])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 4. Training & Evaluation ===
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader, threshold=0.5):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > threshold
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

@torch.no_grad()
def find_best_threshold(loader, thresholds=np.linspace(0, 1, 101)):
    model.eval()
    best_threshold = 0.5
    best_f1 = 0.0

    # Alle Outputs und Labels sammeln, damit man nicht für jeden Threshold neu durch die Daten geht
    all_outs = []
    all_labels = []
    for batch in loader:
        out = model(batch.x, batch.edge_index)
        all_outs.append(torch.sigmoid(out))
        all_labels.append(batch.y)
    all_outs = torch.cat(all_outs)
    all_labels = torch.cat(all_labels)

    for threshold in thresholds:
        preds = all_outs > threshold
        tp = ((preds == 1) & (all_labels == 1)).sum().item()
        fp = ((preds == 1) & (all_labels == 0)).sum().item()
        fn = ((preds == 0) & (all_labels == 1)).sum().item()

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

@torch.no_grad()
def show_some_predictions(loader, n_samples=3, threshold=0.5):
    model.eval()
    all_probs = []
    all_preds = []
    all_labels = []

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        probs = torch.sigmoid(out)
        preds = (probs > threshold).long()
        all_probs.append(probs)
        all_preds.append(preds)
        all_labels.append(batch.y)

    all_probs = torch.cat(all_probs)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    total_samples = len(all_labels)
    indices = random.sample(range(total_samples), k=min(n_samples, total_samples))

    for i, idx in enumerate(indices):
        print(f"Sample {i + 1}:")
        print(f"  True label:      {all_labels[idx].item()}")
        print(f"  Predicted prob:  {all_probs[idx].item():.4f}")
        print(f"  Predicted label: {all_preds[idx].item()}")

# === 5. Training starten ===
for epoch in range(1, 51):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader, threshold=0.5)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

# Nach Training besten Threshold bestimmen
best_threshold, best_f1 = find_best_threshold(test_loader)
print(f"\nBester Threshold: {best_threshold:.3f} mit F1-Score: {best_f1:.3f}")

# Evaluation mit bestem Threshold
acc, prec, rec, f1 = evaluate(test_loader, threshold=best_threshold)
print(f"Evaluation mit bestem Threshold:")
print(f"Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

print("\nEin paar Beispielvorhersagen auf Trainingsdaten mit bestem Threshold:")
show_some_predictions(train_loader, n_samples=3, threshold=best_threshold)



Pos Weight: 7.02


C:\Users\uhewm\AppData\Local\Temp\ipykernel_11436\3901431779.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Epoch 01 | Loss: 0.3385 | Acc: 0.984 | Prec: 0.540 | Rec: 0.818 | F1: 0.651
Epoch 02 | Loss: 0.1416 | Acc: 0.977 | Prec: 0.444 | Rec: 0.904 | F1: 0.595
Epoch 03 | Loss: 0.1263 | Acc: 0.980 | Prec: 0.478 | Rec: 0.915 | F1: 0.628
Epoch 04 | Loss: 0.1151 | Acc: 0.980 | Prec: 0.476 | Rec: 0.917 | F1: 0.627
Epoch 05 | Loss: 0.1093 | Acc: 0.983 | Prec: 0.530 | Rec: 0.902 | F1: 0.668
Epoch 06 | Loss: 0.1050 | Acc: 0.986 | Prec: 0.582 | Rec: 0.874 | F1: 0.698
Epoch 07 | Loss: 0.0992 | Acc: 0.981 | Prec: 0.499 | Rec: 0.912 | F1: 0.645
Epoch 08 | Loss: 0.0964 | Acc: 0.984 | Prec: 0.539 | Rec: 0.902 | F1: 0.675
Epoch 09 | Loss: 0.0984 | Acc: 0.984 | Prec: 0.549 | Rec: 0.901 | F1: 0.682
Epoch 10 | Loss: 0.0938 | Acc: 0.986 | Prec: 0.590 | Rec: 0.887 | F1: 0.709
Epoch 11 | Loss: 0.0928 | Acc: 0.987 | Prec: 0.593 | Rec: 0.886 | F1: 0.710
Epoch 12 | Loss: 0.0919 | Acc: 0.986 | Prec: 0.587 | Rec: 0.894 | F1: 0.709
Epoch 13 | Loss: 0.0914 | Acc: 0.982 | Prec: 0.502 | Rec: 0.920 | F1: 0.650
Epoch 14 | L

In [83]:
print("\nEin paar Beispielvorhersagen auf Trainingsdaten mit bestem Threshold:")
show_some_predictions(train_loader, n_samples=25, threshold=best_threshold)




Ein paar Beispielvorhersagen auf Trainingsdaten mit bestem Threshold:
Sample 1:
  True label:      0
  Predicted prob:  0.0040
  Predicted label: 0
Sample 2:
  True label:      0
  Predicted prob:  0.0221
  Predicted label: 0
Sample 3:
  True label:      0
  Predicted prob:  0.0034
  Predicted label: 0
Sample 4:
  True label:      0
  Predicted prob:  0.0033
  Predicted label: 0
Sample 5:
  True label:      0
  Predicted prob:  0.0035
  Predicted label: 0
Sample 6:
  True label:      0
  Predicted prob:  0.0034
  Predicted label: 0
Sample 7:
  True label:      0
  Predicted prob:  0.0033
  Predicted label: 0
Sample 8:
  True label:      0
  Predicted prob:  0.0033
  Predicted label: 0
Sample 9:
  True label:      0
  Predicted prob:  0.0033
  Predicted label: 0
Sample 10:
  True label:      0
  Predicted prob:  0.0033
  Predicted label: 0
Sample 11:
  True label:      0
  Predicted prob:  0.0035
  Predicted label: 0
Sample 12:
  True label:      0
  Predicted prob:  0.0031
  Predicted

In [91]:
import torch

for file in random.sample(all_files, 1):
    with h5py.File(file, "r") as f:
        grp = f["results"]
        graph_properties = pickle.loads(grp["graph_properties"][()])
        # Tensors erstellen
        nodes = torch.tensor(graph_properties[0])          # Shape [199]
        edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
        coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]
        parental_nodes_hgt_events_corrected = grp.attrs["parental_nodes_hgt_events_corrected"]
        gene_absence_presence_matrix = grp.attrs["gene_absence_presence_matrix"] 

        x_node_features = coords.float().T
        
        # Erstelle einen gerichteten Graphen
        G = nx.DiGraph()
        
        # Füge Knoten hinzu (optional mit Koordinaten als Attribut)
        for i, node_id in enumerate(nodes.tolist()):
            G.add_node(node_id, core_distance = coords[:, i].tolist()[0], allele_distance = coords[:, i].tolist()[1])
        
        # Füge Kanten hinzu
        edge_list = edges.tolist()
        for src, dst in zip(edge_list[0], edge_list[1]):
            G.add_edge(src, dst)

        H = deepcopy(G)
            
        for node in G.nodes():
            children = list(G.predecessors(node))
            if children:
                core_sum = sum(G.nodes[child]['core_distance'] for child in children)
                allele_sum = sum(G.nodes[child]['allele_distance'] for child in children)
                H.nodes[node]['core_distance'] = G.nodes[node]['core_distance'] - core_sum
                H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance'] - allele_sum
            else:
                H.nodes[node]['core_distance'] = G.nodes[node]['core_distance']
                H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance']


        node_features = []
        for node in list(H.nodes):
            core = H.nodes[node].get("core_distance", 0.0)
            allele = H.nodes[node].get("allele_distance", 0.0)
            node_features.append([core, allele])
        coords_modified = torch.tensor(node_features, dtype=torch.float32).T
         
        # Node features: nur die zwei Werte pro Knoten (coords)
        x_node_features_modified = coords_modified.float().T  # Shape [num_nodes, 2]

        # Labels
        theta_gains = torch.tensor(
            [1 if node in grp.attrs["parental_nodes_hgt_events_corrected"] else 0 
             for node in graph_properties[0]],
            dtype=torch.long
        )

        x=x_node_features,       # Node Features [num_nodes, 2]
        edge_index=edges,        # Edge Index [2, num_edges]
        y=theta_gains            # Labels [num_nodes]


# === 1. Modellvorhersagen berechnen ===
model.eval()
with torch.no_grad():
    logits = model(x, edge_index)  # Shape: [num_nodes]
    probs = torch.sigmoid(logits).cpu().numpy()  # Werte zwischen 0 und 1

# Map von Node-ID zu Wahrscheinlichkeit
pred_probs = {i: p for i, p in enumerate(probs)}

# === 2. Visualisierung mit Vorhersagen ===
from pyvis.network import Network
from collections import defaultdict
import networkx as nx

net = Network(height="900px", width="100%", directed=True)
net.set_options("""
{
    "layout": {
      "hierarchical": {
        "enabled": true,
        "direction": "DU",
        "sortMethod": "directed",
        "levelSeparation": 50,
        "nodeSpacing": 200
      }
    },
  "nodes": {
    "shape": "dot",
    "size": 12,
    "font": { "size": 16 }
  },
  "edges": {
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    }
  }
}
""")

G = H  # Graph mit berechneten core/allele Werten

levels = defaultdict(list)
for n in G.nodes:
    level = G.nodes[n].get("level", 0)
    levels[level].append(int(n))

def descendants_count(node):
    return len(nx.descendants(G, node))

for level in sorted(levels.keys()):
    nodes_in_level = levels[level]
    if level == 0:
        leaves_level_0 = [n for n in nodes_in_level if n < 100]
        nodes_sorted = sorted(leaves_level_0)
    else:
        nodes_sorted = nodes_in_level
        
    for n in nodes_sorted:
        core = G.nodes[n].get('core_distance', 0)
        allele = G.nodes[n].get('allele_distance', 0)
        pred = pred_probs.get(n, None)

        # Tooltip mit NN-Wahrscheinlichkeit
        if pred is not None:
            title = f"Core: {core:.2f}, Allele: {allele:.2f}, NN: {pred:.3f}"
            label = f"{n}\n({core:.2f}, {allele:.2f}, p={pred:.2f})"
        else:
            title = f"Core: {core:.2f}, Allele: {allele:.2f}"
            label = f"{n}\n({core:.2f}, {allele:.2f})"

        color = "lightblue"
        if n in parental_nodes_hgt_events_corrected:
            color = "red"
        elif n < 100 and gene_absence_presence_matrix[n] == 1:
            color = "green"
        elif n < 100 and gene_absence_presence_matrix[n] == 0:
            color = "black"

        net.add_node(n, label=label, title=title, color=color, level=level)

# Edges hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

net.show("graph_with_predictions.html", notebook=False)


C:\Users\uhewm\AppData\Local\Temp\ipykernel_11436\1236246542.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
C:\Users\uhewm\AppData\Local\Temp\ipykernel_11436\1236246542.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]


IndexError: tuple index out of range

In [88]:
data = graphs_modified[random.choice(range(len(graphs_modified)))]

data.x

tensor([[0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.000